# **Import Libraries**

In [1]:
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
import torch
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from matplotlib import style
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow.keras import layers as tfkl
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout,LSTM,BatchNormalization, Activation
from sklearn.preprocessing import MinMaxScaler
from keras import callbacks
import os ,datetime,math,time

%matplotlib inline
warnings.filterwarnings('ignore')
#%load_ext tensorboard

# **Load data without outliers**

In [2]:
!gdown 1Td2yUesbc68Nf1Jq4-DNcGo5gWLbHQiK # Load clean dataset
!gdown 19_sOEpLtAJFh9fSqYEnuAquR-9SINaFy
data = pd.read_csv('/content/clean_traffic_dataframe_without_outliers.csv')
index = data['index']
data.drop(['index','src_id','dst_id'], inplace=True, axis=  1)

row = 5
data

Downloading...
From: https://drive.google.com/uc?id=1Td2yUesbc68Nf1Jq4-DNcGo5gWLbHQiK
To: /content/clean_traffic_dataframe_with_outliers.csv
100% 21.8M/21.8M [00:00<00:00, 64.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=19_sOEpLtAJFh9fSqYEnuAquR-9SINaFy
To: /content/clean_traffic_dataframe_without_outliers.csv
100% 21.8M/21.8M [00:00<00:00, 66.4MB/s]


,2005-01-01-00-30,2005-01-01-00-45,2005-01-01-01-00,2005-01-01-01-15,2005-01-01-01-30,2005-01-01-01-45,2005-01-01-02-00,2005-01-01-02-15,2005-01-01-02-30,2005-01-01-02-45,...,2005-01-31-21-30,2005-01-31-21-45,2005-01-31-22-00,2005-01-31-22-15,2005-01-31-22-30,2005-01-31-22-45,2005-01-31-23-00,2005-01-31-23-15,2005-01-31-23-30,2005-01-31-23-45
0,396708.218750,338875.562500,341894.531250,347749.593750,320159.500000,336569.968750,304732.937500,318965.968750,313401.906250,347195.343750,...,311502.937500,317854.750000,315334.906250,320295.812500,318063.656250,320748.468750,298609.437500,297175.156250,305258.218750,310534.281250
1,28093.751953,28485.341797,38402.675781,31250.835938,50818.000000,20076.400391,33443.359375,40365.109375,30642.080078,34432.773438,...,30722.843750,31104.542969,47739.316406,35189.562500,33934.667969,30320.167969,25305.537109,26145.296875,28801.875000,30300.666016
2,16916.560547,11769.341797,5830.630859,7052.648926,6173.021973,4775.511230,5676.648926,4924.951172,4945.306641,5441.315430,...,3370.639893,1122.391113,1256.791138,2634.729004,3196.266602,4596.844238,7102.284180,5395.040039,4077.155518,10089.449219
3,3662.808838,1950.506714,2293.377686,1915.724365,1556.533325,1623.608887,1072.942261,855.422180,2896.497803,3293.386719,...,194.088898,726.826721,4217.244629,4217.244629,4217.244629,8732.426758,62.880001,412.213287,166.906693,119.040001
4,6552.933105,5445.911133,5831.057617,5757.155762,5453.600098,5178.399902,7083.333496,8414.426758,7294.720215,4109.404297,...,2915.893311,2589.146729,4792.293457,4280.568848,3852.213379,2539.466797,2017.928955,1540.204346,1812.302246,1205.466675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,1326.924438,572.444397,533.786682,556.604370,575.795593,628.737793,446.000000,586.044373,411.573303,535.466675,...,208.284393,111.973297,119.324402,182.293304,111.555603,198.826706,123.555603,43.866699,51.306702,935.715576
435,496.471100,493.484406,292.071106,182.488907,267.822205,51.137798,103.173302,135.279999,75.306702,146.284393,...,19.004400,0.880000,8.266700,1.564400,16.835600,14.720000,0.435600,0.462200,13.600000,13.866700
436,3781.964355,888.728882,1063.511108,1211.875610,1142.719971,1020.337830,844.088928,1155.155640,627.786682,561.422180,...,1019.119995,2097.875488,2995.155518,5178.213379,3393.697754,508.044403,977.688904,1867.662231,296.853302,299.715607
437,889.226685,8000.817871,3387.182129,544.746704,453.786713,941.253296,4769.849121,5940.586914,2600.355713,603.751099,...,171.235596,355.911102,450.577789,760.711121,760.711121,695.128906,689.431091,2366.382080,381.626709,228.142197


# **Concatenate all series**

In [3]:
long_series = pd.concat([data[col] for col in data.columns], axis=0)
long_series = long_series.reset_index(drop=True)
print('long_series.shape',long_series.shape)


long_series.shape (1291099,)


# **Normalization**

In [4]:
scaler = StandardScaler()
values  = long_series.values.reshape(-1, 1)
long_series = scaler.fit_transform(values).flatten()
long_series.shape

(1291099,)

# **Create Sequences**

In [5]:
# Create sequences
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length)]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

window_size = 20
X, y = create_sequences(long_series, window_size)

# **Split data set**

In [6]:
split = int(0.9 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# **Run Global Model**

## a) Define hyperparameters

In [7]:
import random
Epochs =30
Batch_size = 32
window_size = 20
period = 96
split_train_test_portion = 0.1
validation_portion = 0.1 # Allocate 10 % of train_data to validation
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

## b) Callbacks

In [8]:
def checkpoint(path,metrics='val_loss',save_weights_only=False):

    root_folder = 'Checkpint_Folder'
    filename = 'cp.keras'
    # Join them to create the full path
    full_path = os.path.join(root_folder,path, filename)
    checkpoint_path = full_path
    checkpoint_dir =os.path.dirname(full_path)
    check =callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                          monitor=metrics,
                                          verbose=1,
                                          save_weights_only = save_weights_only,
                                          save_best_only=True,
                                          mode='auto',
                                          save_freq ='epoch')

    return check
#############################################
def early_stopping(metrics='val_loss'):

    early = callbacks.EarlyStopping(monitor=metrics,
                                                  min_delta=.02,
                                                  patience=3,
                                                  verbose=1,
                                                  mode="auto")

    return early

#############################################
def tensor_board(path):

    logdir = os.path.join("TensorBoardlogs_Folder",path, datetime.datetime.now().strftime("%Y%m%d-%H%M"))
    tensor = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    return tensor

## b) Define Model

### b_1) PositionalEncoding

In [9]:
# Positional Encoding
class PositionalEncoding(layers.Layer):
    def __init__(self, d_model, max_len):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(max_len, d_model)

    def get_config(self):
        config = super().get_config()
        return config

    def positional_encoding(self, max_len, d_model):
        pos = np.arange(max_len)[:, np.newaxis]
        i = np.arange(d_model)[np.newaxis, :]
        angle_rads = pos / np.power(10000, (2 * (i // 2)) / np.float32(d_model))

        sines = np.sin(angle_rads[:, 0::2])
        cosines = np.cos(angle_rads[:, 1::2])

        pos_encoding = np.concatenate([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[np.newaxis, ...]

        return tf.cast(pos_encoding, dtype=tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

### b_2) Transform_model

In [10]:


# Transformer model
def transformer_model(input_shape, num_heads, ff_dim, num_layers):
    inputs = layers.Input(shape=input_shape)

    # Embeddinga
    x = layers.Dense(ff_dim)(inputs)
    x = PositionalEncoding(ff_dim, input_shape[0])(x)

    # Transformer blocks
    for _ in range(num_layers):
        x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=ff_dim)(x, x)
        x = layers.LayerNormalization(epsilon=1e-6)(x)
        x = layers.Dense(ff_dim, activation='relu')(x)
        x = layers.Dense(ff_dim)(x)

    # Global Average Pooling
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(1)(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model

input_shape = (20, 1)
model = transformer_model(input_shape, num_heads=4, ff_dim=64, num_layers=2)

model.compile(optimizer='adam', loss='mse')
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 20, 1)]              0         []                            
                                                                                                  
 dense (Dense)               (None, 20, 64)               128       ['input_1[0][0]']             
                                                                                                  
 positional_encoding (Posit  (None, 20, 64)               0         ['dense[0][0]']               
 ionalEncoding)                                                                                   
                                                                                                  
 multi_head_attention (Mult  (None, 20, 64)               66368     ['positional_encoding[0][0

### b_3) Train Model

In [11]:
# Train the model
path = f"Transformer_on_Originals_for_traffic_{row}_without_outliers"
my_checkpoints = checkpoint(path=path,metrics='val_loss',save_weights_only=False)
my_eraly_stops = early_stopping()
my_tensors =  tensor_board(path=path)
my_defined_callbacks = [my_checkpoints, my_eraly_stops, my_tensors]
start = time.time()
transformer_model = model.fit(X_train, y_train, epochs=Epochs, batch_size=Batch_size, validation_split=validation_portion,callbacks = my_defined_callbacks)
stop = time.time()
training_time = round((stop-start),0)

Epoch 1/30
32680/32681 [============================>.] - ETA: 0s - loss: 0.9217
Epoch 1: val_loss improved from inf to 1.11258, saving model to Checkpint_Folder/Transformer_on_Originals_for_traffic_5_without_outliers/cp.keras
32681/32681 [==============================] - 347s 10ms/step - loss: 0.9217 - val_loss: 1.1126
Epoch 2/30
32678/32681 [============================>.] - ETA: 0s - loss: 0.8867
Epoch 2: val_loss improved from 1.11258 to 1.05238, saving model to Checkpint_Folder/Transformer_on_Originals_for_traffic_5_without_outliers/cp.keras
32681/32681 [==============================] - 337s 10ms/step - loss: 0.8867 - val_loss: 1.0524
Epoch 3/30
32680/32681 [============================>.] - ETA: 0s - loss: 0.8831
Epoch 3: val_loss improved from 1.05238 to 1.03467, saving model to Checkpint_Folder/Transformer_on_Originals_for_traffic_5_without_outliers/cp.keras
32681/32681 [==============================] - 339s 10ms/step - loss: 0.8831 - val_loss: 1.0347
Epoch 4/30
32681/32681 

## b_4) Define Performance

In [12]:
def performance(model_name,prediction_type, operations_on_input, row, y_true, y_pred,training_time):

    cleaned_traffic_df = pd.read_csv('/content/clean_traffic_dataframe_without_outliers.csv')

    # Extract relevant ids from clean_df
    #src_id = cleaned_traffic_df.iloc[row, 0]
    #dst_id = cleaned_traffic_df.iloc[row, 1]
    y_true = np.squeeze(y_true)
    y_pred = np.squeeze(y_pred)

    # Calculate metrics
    mean_squared_error = mse(y_true=y_true, y_pred=y_pred)
    mean_absolute_error = mae(y_true=y_true, y_pred=y_pred)
    mean_absolute_percentage_error = mape(y_true=y_true, y_pred=y_pred)

    # Create a new row of result
    performance_df = pd.DataFrame({
        "Traffic_id":[row],
        "Src_id": '--',
        "Dst_id": '--',
        "Model": [model_name],
        "Prediction_type":[prediction_type],
        "Input":[operations_on_input],
        "Training_time[s]": [training_time],
        "MAE": [mean_absolute_error],
        "MSE": [mean_squared_error],
        "MAPE[%]": [mean_absolute_percentage_error*100],
    })

    # Append the new row to the existing performance_df


    return performance_df

## b_5) Result

In [13]:
# Performance table for all traffics
y_pred = model.predict(X_test)
performance_total_table = performance(model_name='Model_1: Global Transformer',prediction_type='Prediction_on_Original'
                                      , operations_on_input='Without_outliers_normalized', row='All traffics', y_true=y_test, y_pred=y_pred,training_time = training_time)
display(performance_total_table)

4035/4035 [==============================] - 17s 4ms/step


,Traffic_id,Src_id,Dst_id,Model,Prediction_type,Input,Training_time[s],MAE,MSE,MAPE[%]
0,All traffics,--,--,Model_1: Global Transformer,Prediction_on_Original,Without_outliers_normalized,1705.0,0.424669,1.147705,171.887135


In [14]:
# Result for traffic recognized with row
traffic_data = data.iloc[row,:]
values  = traffic_data.values.reshape(-1, 1)
traffic_data = scaler.transform(values).flatten()
X, y = create_sequences(traffic_data, 20)

def performance(model_name,prediction_type, operations_on_input, row, y_true, y_pred,training_time):

    cleaned_traffic_df = pd.read_csv('/content/clean_traffic_dataframe_without_outliers.csv')

    # Extract relevant ids from clean_df
    src_id = cleaned_traffic_df.iloc[row, 1]
    dst_id = cleaned_traffic_df.iloc[row, 2]
    y_true = np.squeeze(y_true)
    y_pred = np.squeeze(y_pred)

    # Calculate metrics
    mean_squared_error = mse(y_true=y_true, y_pred=y_pred)
    mean_absolute_error = mae(y_true=y_true, y_pred=y_pred)
    mean_absolute_percentage_error = mape(y_true=y_true, y_pred=y_pred)

    # Create a new row of result
    performance_df = pd.DataFrame({
        "Traffic_id":[row],
        "Src_id": [src_id],
        "Dst_id": [dst_id],
        "Model": [model_name],
        "Prediction_type":[prediction_type],
        "Input":[operations_on_input],
        "Training_time[s]": [training_time],
        "MAE": [mean_absolute_error],
        "MSE": [mean_squared_error],
        "MAPE[%]": [mean_absolute_percentage_error*100],
    })

    # Append the new row to the existing performance_df


    return performance_df



y_pred = model.predict(X)
performance_traffic_5_table = performance(model_name='Model_1: Global Transformer',prediction_type='Prediction_on_Original'
                                      , operations_on_input='Without_outliers_normalized', row=5, y_true=y, y_pred=y_pred,training_time = training_time)
display(performance_traffic_5_table)

92/92 [==============================] - 0s 4ms/step


,Traffic_id,Src_id,Dst_id,Model,Prediction_type,Input,Training_time[s],MAE,MSE,MAPE[%]
0,5,12.0,18.0,Model_1: Global Transformer,Prediction_on_Original,Without_outliers_normalized,1705.0,0.718665,1.064684,83.243974


In [15]:
traffic_data = data.iloc[73,:]
values  = traffic_data.values.reshape(-1, 1)
traffic_data = scaler.transform(values).flatten()
X, y = create_sequences(traffic_data, 20)

y_pred = model.predict(X)
performance_traffic_73_table = performance(model_name='Model_1: Global Transformer',prediction_type='Prediction_on_Original'
                                      , operations_on_input='Without_outliers_normalized', row=73, y_true=y, y_pred=y_pred,training_time = training_time)
display(performance_traffic_73_table)

92/92 [==============================] - 0s 4ms/step


,Traffic_id,Src_id,Dst_id,Model,Prediction_type,Input,Training_time[s],MAE,MSE,MAPE[%]
0,73,23.0,11.0,Model_1: Global Transformer,Prediction_on_Original,Without_outliers_normalized,1705.0,0.217638,0.047375,73.441463


In [16]:
performance_total_table.to_csv('performance_total_table.csv',index=True)
performance_traffic_5_table.to_csv('performance_traffic_5_table.csv',index=True)
performance_traffic_73_table.to_csv('performance_traffic_73_table.csv',index=True)